# РК 2
## ИУ5-63Б Попов С.
## Вариант 15
## Датасет - U.S. Education Datasets: Unification Project

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('/kaggle/input/us-education-datasets-unification-project/states_all.csv')

In [3]:
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,NaN,NaN,731634.0,208.0,252.0,207.0,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,NaN,NaN,122487.0,NaN,NaN,NaN,NaN
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,NaN,NaN,673477.0,215.0,265.0,209.0,NaN
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,NaN,NaN,441490.0,210.0,256.0,211.0,NaN
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,NaN,NaN,5254844.0,208.0,261.0,202.0,NaN


#### Будем предсказывать доход образовательных центров на основе оценок, полученных студентами, затрат на образование и местоположения

In [4]:
df.dropna(inplace = True)

In [5]:
# Удалим ненужные колонки и те, которые складываются из других
for column_name in df.columns:
    if "REVENUE" in column_name and column_name != "TOTAL_REVENUE":
        df.drop(column_name, axis=1, inplace = True)
df.drop("PRIMARY_KEY", axis=1, inplace = True)
df.drop("TOTAL_EXPENDITURE", axis=1, inplace = True)
df.drop("GRADES_ALL_G", axis=1, inplace = True)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 355 entries, 561 to 1223
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   STATE                         355 non-null    object 
 1   YEAR                          355 non-null    int64  
 2   ENROLL                        355 non-null    float64
 3   TOTAL_REVENUE                 355 non-null    float64
 4   INSTRUCTION_EXPENDITURE       355 non-null    float64
 5   SUPPORT_SERVICES_EXPENDITURE  355 non-null    float64
 6   OTHER_EXPENDITURE             355 non-null    float64
 7   CAPITAL_OUTLAY_EXPENDITURE    355 non-null    float64
 8   GRADES_PK_G                   355 non-null    float64
 9   GRADES_KG_G                   355 non-null    float64
 10  GRADES_4_G                    355 non-null    float64
 11  GRADES_8_G                    355 non-null    float64
 12  GRADES_12_G                   355 non-null    float64
 13  GRADES_

In [7]:
label_encoder = LabelEncoder()
df["STATE"] = label_encoder.fit_transform(df["STATE"])

In [8]:
df.describe()

,STATE,YEAR,ENROLL,TOTAL_REVENUE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,355.000000,355.000000,3.550000e+02,3.550000e+02,3.550000e+02,3.550000e+02,3.550000e+02,3.550000e+02,355.000000,355.000000,355.000000,355.000000,355.000000,3.550000e+02,3.550000e+02,355.000000,355.000000,355.000000,355.000000
mean,25.025352,2008.966197,9.313040e+05,1.097719e+07,5.736849e+06,3.247157e+06,4.941349e+05,1.074615e+06,23098.921127,70791.478873,70882.808451,72152.664789,64666.498592,5.728106e+05,2.849564e+05,238.935211,280.980282,219.943662,263.600000
std,14.725815,3.991373,1.082681e+06,1.317823e+07,7.097473e+06,3.728515e+06,5.780363e+05,1.528286e+06,35065.747998,82271.676298,82338.039862,83304.816013,75240.206360,6.647012e+05,3.313938e+05,6.604123,8.494409,6.996140,6.685679
min,0.000000,2003.000000,4.417900e+04,8.010080e+05,3.306980e+05,2.394350e+05,2.760800e+04,2.007000e+04,399.000000,5190.000000,4643.000000,4328.000000,2971.000000,3.769800e+04,1.682800e+04,205.000000,243.000000,188.000000,238.000000
25%,12.000000,2005.000000,2.700105e+05,2.759076e+06,1.425959e+06,8.519230e+05,1.303250e+05,2.478750e+05,3675.000000,20967.000000,20253.500000,20654.000000,18465.000000,1.649815e+05,8.087650e+04,236.000000,276.000000,216.000000,259.000000
50%,25.000000,2009.000000,6.651090e+05,6.529894e+06,3.380399e+06,2.050683e+06,3.563890e+05,6.182840e+05,13230.000000,52267.000000,50354.000000,50043.000000,41270.000000,4.052620e+05,1.845880e+05,240.000000,282.000000,221.000000,265.000000
75%,38.000000,2013.000000,1.027190e+06,1.298782e+07,6.732184e+06,3.731412e+06,5.773020e+05,1.085066e+06,28993.000000,81085.000000,81431.000000,81467.000000,79046.500000,6.513240e+05,3.290140e+05,243.000000,286.500000,225.000000,268.000000
max,50.000000,2015.000000,6.307022e+06,7.395890e+07,4.195426e+07,2.169368e+07,3.759373e+06,1.022366e+07,249524.000000,506831.000000,493415.000000,500143.000000,498403.000000,3.929869e+06,2.011865e+06,253.000000,301.000000,237.000000,277.000000


In [9]:
df.size

6745

In [10]:
# Проведем масшитабирование данных
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer
sc2 = StandardScaler()
sc2_data = sc2.fit_transform(df)
sc2_data = pd.DataFrame(sc2_data, columns=df.columns)

In [11]:
X = sc2_data
y = sc2_data['TOTAL_REVENUE']

In [12]:
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)     

In [15]:
# Модели
dt_reg = DecisionTreeRegressor(random_state=0)
rf_reg = RandomForestRegressor(n_estimators=1000, random_state=0)

# Обучение моделей
dt_reg.fit(X_train, y_train)
rf_reg.fit(X_train, y_train)

# Предсказания моделей
y_pred_dt = dt_reg.predict(X_test)
y_pred_rf = rf_reg.predict(X_test)

In [16]:
dt_r2 = r2_score(y_test, y_pred_dt)
rf_r2 = r2_score(y_test, y_pred_rf)
dt_mse = mean_squared_error(y_test, y_pred_dt, squared=False)
rf_mse = mean_squared_error(y_test, y_pred_rf, squared=False)

In [18]:
print(f'Дерево решений R^2: {dt_r2:.4f}')
print(f'Случайное дерево R^2: {rf_r2:.4f}')
print()
print(f'Дерево решений MSE: {dt_mse:.4f}')
print(f'Случайное дерево MSE: {rf_mse:.4f}')

Дерево решений R^2: 0.9877
Случайное дерево R^2: 0.9978

Дерево решений MSE: 0.1145
Случайное дерево MSE: 0.0479


Я использовал две метрики: 
- R2 для определения процента данных, объясняемых моделью, а не стохаистически
- MSE для определения квадрата отклонения предсказаний

Набор входных признаков достаточно велик, поэтому вполне ожидаемо, что RandomForest выдал лучшие метрики, так как деревья решений очень чувствительны к признакам, по которым происходит ветвление. \
Высокое значение R2 может объясняться не только качеством модели, но и логичностью самих данных, так как платежеспособность и качество образование в социуме обычно коррелируют.